In [ ]:
import spectracorrm as spc
import os
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import numpy as np
from pybaselines import Baseline
import pandas as pd

In [ ]:
freq = "442" # The theoretical frequency that should be used (example: 442, - as it is in the name of the theoretical file)
experimental_path = f"./experimental-data/{freq}" # Path to the experimental folder
theoretical_path = "./theoretical-data" # Path to the theoretical folders
results_path = f"./results/{freq}" # Where to save the results
basis_sets = ["6311Gdp", "cc-pVTZ", "Def2TZVP"] # Basis sets that were used
methods = ["APFD", "B3LYP", "BMK", "CAM-B3LYP", "M062X", "MN12SX", "MN15L", "PBE0", "wB97XD"] # Methods that were used (e.g. "APFD", "B3LYP", "BMK")
min_scale_factor = 0.8
max_scale_factor = 1.2
points_scale_factor = 400
min_th_freq = 800 # Minimum theoretical spectra frequency in cm^-1
max_th_freq = 1800 # Maximum theoretical spectra frequency in cm^-1
points_th_freq = 1000 # Total number of points for the theoretical spectra
step_th_freq = (max_th_freq - min_th_freq) / points_scale_factor
th_sigma = 24 # half-width at half-maximum (HWHM) in cm^-1

# DO NOT MODIFY BELOW THIS LINE
names = [] # Initialize list of all literal names of the compounds - SHOULD BE LEFT EMPTY IN THIS CELL
names_d = {} # Initialize dictionary for mapping between indexes and compounds codes - SHOULD BE LEFT EMPTY IN THIS CELL
exp_spectra = [] # Initialize list for experimental spectra - SHOUL BE LEFT EMPTY IN THIS CELL
scales = np.linspace(min_scale_factor, max_scale_factor, points_scale_factor) # Scaling factors
scales = np.round(scales, 3) # Round scaling factors to 3 digits
os.makedirs(os.path.join(results_path)) # Create results directory

In [ ]:
spc_files = [s for s in os.listdir(experimental_path) if ".txt" in s]
print(spc_files)

In [ ]:
os.makedirs(os.path.join(results_path, "Experimental"))
for s in spc_files:
    names.append(s.split(".")[0])
    spectra = os.path.join(experimental_path, s)
    obj = spc.Spectrum.from_csv(spectra)
    baseline_fitter = Baseline(obj.frequencies,check_finite=False)
    bkg_1 = baseline_fitter.modpoly(obj.intensities, poly_order=3)[0]
    obj.intensities -= bkg_1
    
    new_x = np.linspace(min_th_freq, max_th_freq, points_th_freq)
    obj.intensities = np.interp(new_x, obj.frequencies, obj.intensities)
    obj.frequencies = np.round(new_x, 0)
    obj.frequencies = np.round(obj.frequencies,0)
    obj.intensities = savgol_filter(obj.intensities, 30, 3)
    obj.normalize()
    obj.plot()
    print(s)
    obj.export_csv(os.path.join(results_path, "Experimental", f"{names[-1]}-processed.txt"))        
    exp_spectra.append(obj)
    
for index, name in enumerate(names):
    names_d[index] = name

In [ ]:
for basis in basis_sets:
    for method in methods:
        for compound in names_d.values():
            export_path = os.path.join(results_path, "Theoretical", basis, method, compound)
            os.makedirs(export_path, exist_ok=True)
            full_path_to_csv = os.path.join(theoretical_path, freq, basis, method, "Raman-Spectra", f"{compound}-{freq}.txt")
            print(full_path_to_csv)
            try:
                df = pd.read_csv(full_path_to_csv, header=None)
            except FileNotFoundError:
                print(f"FNFE {full_path_to_csv}")
            except:
                print(f"ERROR {full_path_to_csv}")
            freqlist = df[0].tolist()
            intlist = df[1].tolist()
            for scale in scales:
                if not os.path.exists(os.path.join(export_path,f"{compound}-{freq}-{scale}.csv")):
                    try:
                        t = spc.TheoreticalSpectrum(freqlist, intlist, min_th_freq, max_th_freq, step_th_freq, sigma=th_sigma, scale=scale, raman=True)
                        t.export_csv(os.path.join(export_path,f"{compound}-{freq}-{scale}.csv"))
                    except:
                        print(f"ERROR: {full_path_to_csv}")
                        break
                else:
                    pass

In [ ]:
def compare_spc(compound):
    global basis_sets
    global methods
    global freq
    global names_d
    fname = os.path.join(results_path, "Theoretical", "Raman_PearsonR")
    os.makedirs(fname, exist_ok=True)
    d = {}
    
    for basis in basis_sets:
        for method in methods:
            folder = os.path.join(results_path, "Theoretical", basis, method, names_d[compound])
            if os.path.exists(os.path.join(fname,f"{names_d[compound]}-{basis}-{method}-{freq}.csv")):
                print(f"Skip {names_d[compound]}-{basis}-{method}-{freq}.csv")
                continue
            for scale in scales:
                name = f"{compound}_{scale}"
                obj = spc.Spectrum.from_csv(os.path.join(folder,f"{names_d[compound]}-{freq}-{scale}.csv"))
                obj.frequencies = np.array(obj.frequencies)
                obj.intensities = np.array(obj.intensities)
                obj.normalize()
                d[scale] = exp_spectra[compound].compare(obj)
    
            df = pd.DataFrame.from_dict(d, orient='index', columns= ['R'])
            df.index.name = "Scale Factor"
            df.to_csv(os.path.join(fname,f"{names_d[compound]}-{basis}-{method}-{freq}.csv"))

In [ ]:
for compound in names_d.keys():
    compare_spc(compound)

In [ ]:
def calc_average():
    global names_d
    global freq
    fname = os.path.join(results_path, "Theoretical", "Raman_PearsonR")
    export_path = os.path.join(fname, "Average")
    os.makedirs(export_path, exist_ok=True)
    for basis in basis_sets:
        for method in methods:
            dflist = []
            for compound in names_d.keys():
                with open(os.path.join(fname, f"{names_d[compound]}-{basis}-{method}-{freq}.csv")) as file:
                    df = pd.read_csv(file)
                    dflist.append(df["R"])
                    res = pd.DataFrame({"Scale Factor" : list(df["Scale Factor"]),
                                        "Average R" : list(sum(dflist) / 5)})
                    res.to_csv(os.path.join(export_path, f"{basis}-{method}.txt",), index=False)

In [ ]:
calc_average()

In [ ]:
def get_best_correlation():
    global names_d
    global freq
    fname = os.path.join(results_path, "Theoretical", "Raman_PearsonR")
    for basis in basis_sets:
        for method in methods:
            with open(os.path.join(fname, "Average", f"{basis}-{method}.txt")) as file:
                df = pd.read_csv(file)
                best_corr = df.loc[df["Average R"].idxmax()]
                print(f"{basis}-{method}\n")
                print(best_corr.to_string())
                print("\n")

In [ ]:
get_best_correlation()